In [0]:
%sql
/*
NoteBook que simula el Job JP_PATRIF_GAR_0500_INS_ARCHIVO Que solamente genera la insercion de datos de hacia CIERREN.TTAFOGRAL_ARCHIVO del ambiente de OCI
para las solicitudes rechazadas de SUBPROCESO 364, 365, 368
*/

In [0]:
%run "./startup" 

In [0]:
params = WidgetParams(
    {
        "sr_proceso": str,
        "sr_subproceso": str,
        "sr_subetapa": str,
        "sr_folio": str,
        "sr_instancia_proceso": str,
        "sr_usuario": str,
        "sr_etapa": str,
        "sr_id_snapshot": str,
        "sr_id_archivo": str,
        "sr_fec_arc": str,
        "sr_paso": str,
        "sr_id_archivo_06": str,
        "sr_id_archivo_09": str
    }
)
params.validate()
conf = ConfManager()
query = QueryManager()
db = DBXConnectionManager()
queries_df = query.get_sql_list()
# filtar solo los registros donde la columan 'Archivo SQL' comiencen por el nombre del notebook
display(queries_df.filter(col("Archivo SQL").startswith("GAR")).orderBy("Archivo SQL"))

In [0]:
from datetime import datetime
import pytz

# Inicializa la clase para subir los archivos
file_manager = FileManager(err_repo_path=conf.err_repo_path)

# Define the timezone for Mexico
mexico_tz = pytz.timezone("America/Mexico_City")

# Genera la fecha actual en el formato YYYYMMDD
fecha_actual = datetime.now(mexico_tz).strftime("%Y%m%d")
#fecha timeStamp
fecha_actual_sin_formato = datetime.now(mexico_tz).strftime("%d-%m-%Y %H:%M:%S.%f")

In [0]:
statement_001 = query.get_statement(
"GAR_0500_INS_ARCHIVO_RECH.sql",
SR_FOLIO=params.sr_folio,
SR_PROCESO=params.sr_proceso,
SR_SUBPROCESO=params.sr_subproceso,
SR_FECHA_ACTUAL=fecha_actual,
SR_TIMESTAMP=fecha_actual_sin_formato,
SR_PREFIJO_ARCH_TA_364=conf.prefijo_arch_ta_364,
SR_NOM_ARCHIVO_RECH_TA_364=conf.nombre_arch_recha_ta_364,
SR_PREFIJO_ARCH_AG_365=conf.prefijo_arch_ag_365,
SR_NOM_ARCHIVO_RECH_AG_365=conf.nom_archivo_rech_ag_365,
SR_PREFIJO_ARCH_UG_368=conf.prefijo_arch_ug_368,
SR_NOM_ARCHIVO_RECH_UG_368=conf.nombre_arch_recha_ug_368,
SR_USUARIO=params.sr_usuario,
SR_EXT_TXT=conf.ext_txt
)

In [0]:
if conf.debug:
    df = db.read_data("default", statement_001)
if conf.debug:
    display(df)

In [0]:
temp_delta = 'DELTA_0500_INS_ARCHIVO_RECHAZADOS_'+ params.sr_subproceso + '_' + params.sr_folio
db.write_delta(temp_delta, db.read_data("default", statement_001), "overwrite")
if conf.debug:
    display(db.read_delta(temp_delta))

In [0]:

df_inserta_archivo = db.read_delta(f"DELTA_0500_INS_ARCHIVO_RECHAZADOS_{params.sr_subproceso}_{params.sr_folio}")\
.select("FTC_NOMBRE","FCN_ID_PROCESO","FCN_ID_SUBPROCESO","FTD_FEH_GEN","FTD_FEH_CRE","FTC_USU_CRE","FCN_ID_ESTATUS_ARCHIVO")

#Se elimina a petición de servicios el campo FTD_FEH_ARCHIVO

df_inserta_archivo_folio = db.read_delta(f"DELTA_0500_INS_ARCHIVO_RECHAZADOS_{params.sr_subproceso}_{params.sr_folio}")\
.select("FTN_ID_ARCHIVO","FTC_FOLIO")

#df_from_delta_001=db.read_delta(f"demo_delta_001_{params.sr_subproceso}_{params.sr_folio}")
if conf.debug:
  display(df_inserta_archivo)
  display(df_inserta_archivo_folio)

In [0]:
#Carga los datos del primer dataframe a la tabla CIERREN.TTAFOGRAL_ARCHIVO
db.write_data(df_inserta_archivo, "CIERREN.TTAFOGRAL_ARCHIVO", "default", "append")



In [0]:
#Carga los datos del segundo dataframe a la tabla CIERREN.TRAFOGRAL_ARCHIVO_FOLIO
db.write_data(df_inserta_archivo_folio,"CIERREN.TRAFOGRAL_ARCHIVO_FOLIO", "default", "append")

In [0]:
CleanUpManager.cleanup_notebook(locals())